# Indeterminate Beam calculator

### Table of Contents
1. Initialisation
2. Basic usage (Readme example)
    - 2(b). Additional features (Readme example)
3. Support class breakdown
4. Load classes breakdown
5. Statically determinate beam (Ex 12.14 Hibbeler)
6. Statically Indeterminate beam (Ex 12.21 Hibbeler)
7. Spring supported beam deflection (Ex 12.16 Hibbeler)
8. Axial Loaded Indeterminate Beam (Ex 4.13 Hibbeler)

Note:
You must always run the initialization cell first. 
Then you are free to move to any of the other sections as per the table of contents.

 ## 1. INITIALIZATION


In [ ]:
# RUN THIS CELL FIRST TO INITIALISE GOOGLE NOTEBOOK!!!!
!pip install indeterminatebeam
%matplotlib inline

# import beam and supports
from indeterminatebeam import Beam, Support

# import loads (all load types imported for reference)
from indeterminatebeam import (
    PointTorque,
    PointLoad,
    PointLoadV,
    PointLoadH,
    UDL,
    UDLV,
    UDLH,
    TrapezoidalLoad,
    TrapezoidalLoadV,
    TrapezoidalLoadH,
    DistributedLoad,
    DistributedLoadV,
    DistributedLoadH
)

# Note: load ending in V are vertical loads
# load ending in H are horizontal loads
# load not ending in either takes angle as an input (except torque)


In [1]:
# NOT FOR USE ONLINE.
# Use this instead for initialization if running directly from package folder,
import sys, os
sys.path.insert(0, os.path.abspath('../../'))

# import beam and supports
from indeterminatebeam import Beam, Support

# import loads (all load types imported for reference)
from indeterminatebeam.loading import (
    PointTorque,
    PointLoad,
    PointLoadV,
    PointLoadH,
    UDL,
    UDLV,
    UDLH,
    TrapezoidalLoad,
    TrapezoidalLoadV,
    TrapezoidalLoadH,
    DistributedLoad,
    DistributedLoadV,
    DistributedLoadH
)

# Note: load ending in V are vertical loads
# load ending in H are horizontal loads
# load not ending in either takes angle as an input (except torque)


## 2. Basic Usage (Readme example)

In [2]:
##arbritrary example defined in README.md
beam = Beam(7)                          # Initialize a Beam object of length 9m with E and I as defaults
beam_2 = Beam(9,E=2000, I =100000)      # Initializa a Beam specifying some beam parameters

a = Support(5,(1,1,0))                  # Defines a pin support at location x = 5m  
b = Support(0,(0,1,0))                  # Defines a roller support at location x = 0m
c = Support(7,(1,1,1))                  # Defines a fixed support at location x = 7m
beam.add_supports(a,b,c)    

load_1 = PointLoadV(1,2)                # Defines a point load of 1kn acting up, at location x = 2m
load_2 = DistributedLoadV(2,(1,4))      # Defines a 2kN UDL from location x = 1m to x = 4m 
load_3 = PointTorque(2, 3.5)            # Defines a 2kN.m point torque at location x = 3.5m
beam.add_loads(load_1,load_2,load_3)    # Assign the support objects to a beam object created earlier

beam.analyse()

fig_1 = beam.plot_beam_external()
fig_1.show()

fig_2 = beam.plot_beam_internal()
fig_2.show()

fig_1.write_image("./readme_example_diagram.png")
fig_2.write_image("./readme_example_internal.png")
fig_1.write_html("./readme_example_diagram.html")
fig_2.write_html("./readme_example_internal.html")

## 2(b). Additional Features (Readme example)

In [ ]:
# Run section 2 (prior to running this example)   

##query for the data at a specfic point
print("bending moments at 3m: " + str(beam.get_bending_moment(3)))
print("shear forces at 1,2,3,4,5m points: " + str(beam.get_shear_force(1,2,3,4,5)))
print("normal force absolute max: " + str(beam.get_normal_force(return_absmax=True)))
print("deflection max: " + str(beam.get_deflection(return_max = True)))   

##add a query point to a plot (adds values on plot)
beam.add_query_points(1,3,5)
beam.remove_query_points(5)

## plot the results for the beam
fig = beam.plot_beam_internal()
fig.show()

## 3. Support class breakdown

In [ ]:
# The parameters for a support class are as below, taken from the docstring
# for the Support class __init__ method.

# Parameters:
#         -----------
#         coord: float
#             x coordinate of support on a beam (default 0)
#         fixed: tuple of 3 booleans
#             Degrees of freedom that are fixed on a beam for movement in
#             x, y and bending, 1 represents fixed and 0 represents free
#             (default (1,1,1))
#         kx :
#             stiffness of x support (kN/mm), if set will overide the
#             value placed in the fixed tuple. (default = None)
#         ky : (positive number)
#             stiffness of y support (kN/mm), if set will overide the
#             value placed in the fixed tuple. (default = None)


# Lets define every possible degree of freedom combination for
# supports below, and view them on a plot:
support_0 = Support(0, (1,1,1))     # conventional fixed support
support_1 = Support(1, (1,1,0))     # conventional pin support
support_2 = Support(2, (1,0,1))     
support_3 = Support(3, (0,1,1))
support_4 = Support(4, (0,0,1))
support_5 = Support(5, (0,1,0))     # conventional roller support
support_6 = Support(6, (1,0,0))

# Note we could also explicitly define parameters as follows:
support_0 = Support(coord=0, fixed=(1,1,1))

# Now lets define some spring supports
support_7 = Support(7, (0,0,0), kx = 10)    #spring in x direction only
support_8 = Support(8, (0,0,0), ky = 5)     # spring in y direction only
support_9 = Support(9, (0,0,0), kx = 100, ky = 100)     # spring in x and y direction

# Now lets define a support which is fixed in one degree of freedom
# but has a spring stiffness in another degree of freedom
support_10 = Support(10, (0,1,0), kx = 10) #spring in x direction, fixed in y direction
support_11 = Support(11, (0,1,1), kx = 10) #spring in x direction, fixed in y and m direction

# Note we could also do the following for the same result since the spring
# stiffness overides the fixed boolean in respective directions
support_10 = Support(10, (1,1,0), kx =10)

# Now lets plot all the supports we have created
beam = Beam(11)

beam.add_supports(
    support_0,
    support_1,
    support_2,
    support_3,
    support_4,
    support_5,
    support_6,
    support_7,
    support_8,
    support_9,
    support_10,
    support_11,
)

fig = beam.plot_beam_diagram()
fig.show()

## 4. Load classes breakdown

### 4.1 Point Torque

In [ ]:
# defined using a force (technically a moment, however force is used to maintain consistenct for all load classes) and a coordinate. An anti-clockwise moment is positive by convention of this package.
load_1 = PointTorque(force=1, coord=1)
load_2 = PointTorque(force=-1, coord=2)

# Plotting the loads
beam = Beam(3)
beam.add_loads(
    load_1,
    load_2
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.2 Point Load

In [ ]:
# defined by force, coord and angle (0)
load_1 = PointLoad(force=1, coord=1, angle=0)
load_2 = PointLoad(force=1, coord=2, angle=45)
load_3 = PointLoad(force=1, coord=3, angle=90)

# Plotting the loads
beam = Beam(4)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.3 Uniformly Distributed Load (UDL)

In [ ]:
# defined by force, span (tuple with start and end point) 
# and angle of force
load_1 = UDL(force=1, span=(1,2), angle = 0)
load_2 = UDL(force=1, span=(3,4), angle = 45)
load_3 = UDL(force=1, span=(5,6), angle = 90)

# Plotting the loads
beam = Beam(7)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.4 Trapezoidal Load

In [ ]:
# defined by force (tuple with start and end force), 
# span (tuple with start and end point) and angle of force
load_1 = TrapezoidalLoad(force=(1,2), span=(1,2), angle = 0)
load_2 = TrapezoidalLoad(force=(-1,-2), span=(3,4), angle = 45)
load_3 = TrapezoidalLoad(force=(-1,2), span=(5,6), angle = 90)

# Plotting the loads
beam = Beam(7)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.5 Distributed Load

In [ ]:
# defined with Sympy expression of the distributed load function 
# expressed using variable x which represents the beam x-coordinate. 
# Requires quotation marks around expression. As with the UDL and 
# Trapezoidal load classes other parameters to express are the span 
# (tuple with start and end point) and angle of force.
# NOTE: where UDL or Trapezoidal load classes can be spefied (linear functions)
# they should be used for quicker analysis times.

load_1 = DistributedLoad(expr= "2", span=(1,2), angle = 0)
load_2 = DistributedLoad(expr= "2*(x-6)**2 -5", span=(3,4), angle = 45)
load_3 = DistributedLoad(expr= "cos(5*x)", span=(5,6), angle = 90)

# Plotting the loads
beam = Beam(7)
beam.add_loads(
    load_1,
    load_2,
    load_3
)

fig = beam.plot_beam_diagram()
fig.show()

### 4.6 Vertical and Horizontal load child classes

In [ ]:
# for all loads except the point torque an angle is specified for the
# direction of the load. If the load to be specified is to be completely
# vertical or completely horizontal a V (vertical) or a H (horizontal)
# can be added at the end of the class name, and the angle does then
# not need to be spefied.

# The following two loads are equivalent horizontal loads
load_1 = PointLoad(force=1, coord=1, angle = 0)
load_2 = PointLoadH(force=1, coord=2)

# The following two loads are equivalent vertical loads
load_3 = PointLoad(force=1, coord=3, angle = 90)
load_4 = PointLoadV(force=1, coord=4)

# The following two loads are also equivalent (a negative sign
# esentially changes the load direction by 180 degrees).
load_5 = PointLoad(force=1, coord=5, angle = 0)
load_6 = PointLoad(force=-1, coord=6, angle = 180)


# Plotting the loads
beam = Beam(7)
beam.add_loads(
    load_1,
    load_2,
    load_3,
    load_4,
    load_5,
    load_6
)

fig = beam.plot_beam_diagram()
fig.show()

## 5. STATICALLY DETERMINATE BEAM (Ex 12.14 Hibbeler)

In [ ]:
## Statically Determinate beam (Ex 12.14 Hibbeler)
## Determine the reactions at the roller support B of the beam described below: 
## 3m long, fixed at A (x = 0m), roller support at B (x=3m), vertical point load at midpan of 8kN, UDL of 6kN/m, EI constant.

beam = Beam(8, E=1, I = 10**12)     ##EI Defined to be 1 kN.m2 to get the deflection as a function of EI

a = Support(0, (1,1,1))             ##explicitly stated although this is equivalent to Support() as the defaults are for a cantilever on the left of the beam.

load_1 = TrapezoidalLoadV((-4,0),(0,6))

beam.add_supports(a)
beam.add_loads(load_1)

beam.analyse()
print(f"Deflection is {beam.get_deflection(8)} kN.m3 / EI ")

fig = beam.plot_beam_internal()  
fig.show()
# Note: all plots are correct, deflection graph shape is correct but for actual deflection values will need real EI properties.

##save the results as a pdf (optional)
# Can save figure using `fig.write_image("./results.pdf")` (can change extension to be
# png, jpg, svg or other formats as reired). Requires pip install -U kaleido


## 6. STATICALLY INDETERMINATE BEAM (Ex 12.21 Hibbeler)

In [ ]:
# Statically Indeterminate beam (Ex 12.21 Hibbeler)
# Determine the reactions at the roller support B of the beam described below: 
# 3m long, fixed at A (x = 0m), roller support at B (x=3m), vertical point load at midpan of 8kN, UDL of 6kN/m, EI constant.

beam = Beam(3)

a = Support(0,(1,1,1))  
b = Support(3,(0,1,0))

load_1 = PointLoadV(-8,1.5)
load_2 = UDLV(-6, (0,3))

beam.add_supports(a,b)
beam.add_loads(load_1,load_2)

beam.analyse()

print(f"The beam has an absolute maximum shear force of: {beam.get_shear_force(return_absmax=True)} kN")
print(f"The beam has an absolute maximum bending moment of: {beam.get_bending_moment(return_absmax=True)} kN.m")
print(f"The beam has a vertical reaction at B of: {beam.get_reaction(3,'y')} kN")

fig1 = beam.plot_beam_external()  
fig1.show()

fig2 = beam.plot_beam_internal()  
fig2.show()

# fig1.write_image("./example_1_external.png")
# fig2.write_image("./example_1_internal.png")




## 7. SPRING SUPPORTED BEAM (Ex 12.16 Hibbeler)

In [ ]:
## Spring Supported beam (Ex 12.16 Hibbeler)
## Determine the vertical displacement at x = 1m for the beam detailed below:
## 3m long, spring of ky = 45kN/m at A (x = 0 m) and B (x = 3 m), vertical point load at x = 1m of 3 kN, E = 200GPa, I = 4.6875*10**-6 m4.

##NOTE: BEWARE OF UNITS

##when initializing beam we should specify E and I. Units should be expressed in MPa (N/mm2) for E, and mm4 for I
beam = Beam(3, E=(200)*10**3, I=(4.6875*10**-6)*10**12)

##creating supports, note that an x support must be specified even when there are no x forces. This will not affect the accuracy or reliability of results.
##Also note that ky units are kN/m in the problem but must be in kN/mm for the program to work correctly.
a = Support(0, (1,1,0), ky = 45*10**-3)   
b = Support(3, (0,1,0), ky = 45*10**-3)

load_1 = PointLoadV(-3,1)

beam.add_supports(a,b)
beam.add_loads(load_1)

beam.analyse()

beam.get_deflection(1)

fig1 = beam.plot_beam_external()  
fig1.show()

fig2 = beam.plot_beam_internal()  
fig2.show()

##results in 38.46 mm deflection ~= 38.4mm specified in textbook (difference only due to their rounding)
##can easily check reliability of answer by looking at deflection at the spring supports. Should equal F/k.
## ie at support A (x = 0 m), the reaction force is 2kN by equilibrium, so our deflection is F/K = 2kn / 45*10-3 kN/mm = 44.4 mm (can be seen in plot)

## 8. AXIAL LOADED INDETERMINATE BEAM (Ex 4.13 Hibbeler)

In [ ]:
##AXIAL LOADED INDETERMINATE BEAM (Ex 4.13 Hibbeler)
## A rod with constant EA has a force of 60kN applied at x =0.1, and the beam has fixed supports at x=0, and x =0.4. Determine the reaction forces.

beam = Beam(0.4)

a = Support()
b = Support(0.4)

load_1 = PointLoadH(-60, 0.1)

beam.add_supports(a,b)
beam.add_loads(load_1)

beam.analyse()
beam.plot_normal_force()
